<a href="https://colab.research.google.com/github/hsm4680/Overall-projects/blob/master/Image_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# google colab의 드라이브를 불러와 코드에 반영하기 위한 드라이브 마운트 작업
# [보기] > [목차]를 선택하면 아래 코드가 자동 삽입되나 지금은 구글 드라이브와 연동하여 gdrive 사용

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# os 모듈은 Operation System의 약자로, 운영체제에서 제공되는 기능을 파이썬에서 수행할 수 있게 함.
# 파이썬을 이용해 파일을 복사하거나 디렉토리를 생성하고, 특정 디렉토리 내의 파일을 불러오는 등으로 사용됨.

import os

# os.chdir 함수는 R에서 setwd()와 동일한 역할을 하여 현재 디렉토리를 ''로 변경함.
os.chdir('/content/gdrive/My Drive/Colab Notebooks/')
# os.getcwd()는 현재 디렉토리(Current Working Directory)의 경로를 출력함.
current_path = os.getcwd()
# os.path.join()은 현재 디렉토리의 절대경로/catdog을 출력함.
main_dir = os.path.join(current_path, 'catdog')
# os.makedirs는 지정된 경로에 맞게 디렉토리를 생성함.
# os.mkdir()는 새로운 하위 폴더 하나만을 생성할 수 있음.
# os.makedirs()는 경로에 지정된 모든 폴더를 생성함.
# exist_ok = True는 해당 경로에 폴더가 실제로 존재하는 경우에는 그냥 넘어가고, 없는 경우에만 생성하게 함.
os.makedirs(main_dir, exist_ok=True)


In [ ]:
# numpy, pandas, matplotlib은 파이썬 데이터 분석의 3대장 격으로 각각 np, pd, plt의 명칭으로 임포트함.

# numpy는 데이터 분석을 용이하게 하기 위한 ndarray(np.array) 구조를 지원하여 행렬 연산을 가능하게 함.
import numpy as np # linear algebra
# pandas는 파이썬 데이터 분석에 필수적인 패키지로 시리즈, 데이터프레임, 패널 데이터 형태를 지원함.
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# matplotlib는 파이썬의 시각화 기능을 주로 담당하는 패키지로 거의 모든 형태의 플롯 생성이 가능함.
import matplotlib.pyplot as plt

# opencv(패키지 이름은 cv2)는 영상처리 및 컴퓨터 비전과 관련된 대표적인 패키지임.
# 이미지 읽기, 색공간 변환, 리사이징, 출력 및 저장 기능을 담당함.
import cv2

# tensorflow는 대표적인 머신러닝 라이브러리로, 파이썬 이외에도 다양한 언어를 지원함.
# PyTorch와 비교했을 때, PyTorch는 단시간 내 실행해야 하는 프로젝트, Tensorflow는 규모가 큰 프로젝트에 적합함.
import tensorflow as tf

# Keras는 Tensorflow 위에서 동작하는 라이브러리로, 사용자 친화적으로 개발되어 Tensorflow보다 사용이 간편함.
# 단순한 작업이나 기존에 갖춰진 기능을 활용할 경우 Keras, 전문 지식에 깁나한 디테일한 조작은 Tensorflow를 활용함.

# 일반적인 다층 퍼셉트론 신경망, 특히 이미지 분석은 단계적으로 입력층 - 은닉층 - 출력층을 쌓아나감.
# Sequential은 위처럼 model.add()에 기반한 단계적인 층 쌓기에 적합한 방법론임.
from tensorflow.keras.models import Sequential

# Dense : 모든 노드 사이에 연결선이 형성되는 Fully Connected Layer인 Dense 레이어를 사용하게 하는 함수
# Flatten : 2차원 데이터를 Fully Connected Layer에 적용할 수 있도록 1차원으로 바꾸는 함수
# Dropout : 어떤 레이어의 모든 노드에서 나가는 Activation을 제거하여 과적합을 방지하는 함수
# Activation : 활성화 함수를 설정할 수 있게 하는 함수
# Conv2D : 필터를 활용해 특징만 뽑아내는 Convolution 레이어를 사용하게 하는 함수
# MaxPooling2D : 사소한 변화는 무시하는 Max Pooling 레이어를 사용하게 하는 함수
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D

In [ ]:
# Image 클래스는 Keras 라이브러리의 Preprocessing 모듈에 포함되어 있음.
# 훈련 데이터 또는 평가 데이터로 이미지 파일을 이용하기 위해 불러올 때 이용됨.
from keras.preprocessing.image import * 
# Keras에 기반해 분류 문제를 해결하기 위해 to_categorical을 불러옴.
from tensorflow.keras.utils import to_categorical
# 데이터를 훈련 데이터와 평가 데이터로 구분하기 위해 이용함.
from sklearn.model_selection import train_test_split

# Random 모듈은 random() 함수로 대표되며 난수 생성에 주로 이용됨.
import random


In [ ]:
# 사전 훈련된 딥러닝 모델을 사용하는 전이 학습(Transfer Learning)을 시행하는 경우,
# 모델에 기존에 정의된 특정한 크기의 입력을 요구하는 경우가 많음.
# 따라서 입력될 이미지의 크기를 그에 맞게 조절할 필요가 있음.

IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNEL=3

In [ ]:
filenames = os.listdir("/content/gdrive/My Drive/Colab Notebooks/catdog/train/train/")

# 이미지 파일만 있으므로 아직 어떤 것이 고양이인지, 개인지 구분하는 문제를 해결할 수 없음.
# cat and dog 파일은 cat.~~~.jpg 또는 dog.~~~.jpg의 형태임.
# 따라서 앞의 cat과 dog을 더미변수로 코딩하면 분류 문제의 답지를 만들 수 있음.
# for, while 반복문을 위해 전역 수준에서 리스트를 선언하고 초기화함.
# 이 categories에는 1 또는 0이 들어가게 될 것임.

categories=[]
# filenames는 train 폴더 안에 있는 모든 파일의 이름을 Element로 포함하는 리스트임.
# 따라서 모든 파일의 수만큼 반복문을 돌리게 될 것.
for filename in filenames:
  # filename에서 첫번째로 등장하는 .을 기준으로 split(cat.10031 -> cat, 10031)하고, 앞의 것만 선택함 (cat)
    category=filename.split(".")[0]
    # 선택된 category가 dog이면 1, cat이면 0을 부여함.
    if category =="dog":
        categories.append(1)
    else:
        categories.append(0)
# 결과를 pandas 데이터 프레임으로 저장하며, 변수명은 filename과 category로 함.
# 내부 요소는 딕셔너리로 개별 키와 개별 값을 요구하는 것 같지만, 데이터 프레임은 변수명과 데이터로 구분됨.
# 따라서 1개의 키를 변수명으로 지정하면 우리가 알고 있는 R의 데이터 프레임처럼 사용할 수 있음.
df=pd.DataFrame(
    {"filename":filenames,
    "category":categories}
)        
# 작업이 잘 이루어졌는지 확인하기 위해 초기 5개의 데이터를 불러옴.
df.head()

,filename,category
0,cat.10031.jpg,0
1,cat.10039.jpg,0
2,cat.10036.jpg,0
3,cat.10038.jpg,0
4,cat.10037.jpg,0


In [ ]:
# 그래프를 그릴 시 별도의 plt.show() 명령어 없이도 브라우저 상에 그래프를 즉시 출력함.
%matplotlib notebook

# df의 category 컬럼에 대해 value_counts()를 적용해 unique value의 카운팅 결과를 얻고, 이를 막대그래프로 시각화함.
df['category'].value_counts().plot.bar()

<IPython.core.display.Javascript object>

In [ ]:
%matplotlib notebook
# random 모듈의 choice 함수는 내부의 리스트에서 아무 Element나 뽑아냄.
sample = random.choice(filenames)
# Keras의 Preprocessing 모듈에 속한 image 클래스는 load_img로 이미지 파일을 불러올 수 있음.
image = load_img("/content/gdrive/My Drive/Colab Notebooks/catdog/train/train/"+sample)
# matplotlib의 imshow(Image Show)
plt.imshow(image)

<IPython.core.display.Javascript object>

In [ ]:
from keras.models import Sequential
from keras.layers import *

In [ ]:
# 합성곱 신경망을 본격적으로 구성하는 부분.
# 합성곱 신경망은 전반부인 이미지의 특징을 추출하는 부분과 후반부인 클래스를 분류하는 부분으로 나뉨.
# 전반부는 Convolutional Layer와 Pooling Layer를 여러 겹 쌓는 형태로 구성됨.
# Convolutional Layer는 입력 데이터에 필터를 적용한 후 활성화 함수를 통해 출력함.
# Pooling Layer

# 레이어 1
# Sequential()을 모델로 선언했으므로 이후 레이어들은 모두 순차적으로 결합됨.
model = Sequential()
# 첫 번째 레이어는 Convolutional 레이어로, Flatten되기 전 합성곱 신경망의 레이어에 해당함.
# 32 = 필터의 수, (5, 5) = 커널의 행과 열
# activation = "relu" : rectifier 함수로, 은닉층에 주로 쓰임.
model.add(Conv2D(32, (3,3), activation="relu", input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH , IMAGE_CHANNEL)))
# 배치 정규화(BatchNormalization())는 인공신경망 입력값을 평균 0, 분산 1로 정규화해 학습을 촉진함.
# 배치(Batch) : 경사하강법 적용 시 모델이 입력받는 데이터의 청크
model.add(BatchNormalization())
# 출력이 행렬 형태로 나올 경우 그 중 하나의 값을 택하여 구조를 축소하는 방식.
# Max Pooling(최대값 추출)이 가장 자주 사용되며, 간혹 Average Pooling(평균값 추출)도 사용됨.
model.add(MaxPooling2D(pool_size=(2,2)))
# 모델의 구조가 복잡해질수록 과적합의 문제가 일어나므로, 확률적으로 Activation을 드랍하게 함.
model.add(Dropout(0.25))

# 레이어 2 
model.add(Conv2D(64, (3,3), activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# 레이어 3
model.add(Conv2D(128, (3,3), activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# 위의 Convolutional Layer, Pooling Layer를 반복적으로 거치며 주요 특징이 추출됨.
# 이제 비로소 이 특징들만 전결합층에 전달해 학습을 시작하게 할 것임.

# Fully Connected 
# 전결합층으로 데이터를 전달하기 위해서는 이미지의 2차원 자료를 1차원 자료로 변환해야 함.
# Flatten()은 이러한 역할을 수행하는 함수임.
model.add(Flatten())
# 512개의 노드로 구성된 전결합 은닉층 생성, 활성화 함수는 여전히 rectifier
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
# 일반적인 딥러닝 신경망에서 드랍아웃은 0.5라고 함. (이미지에서는 왜 .25를 썼는지 확인 필요)
model.add(Dropout(0.5))

# 마지막 최종 출력층
# 개와 고양이의 이진 분류이므로 2개의 노드만 생성함. 마지막 활성화 함수는 Softmax
model.add(Dense(2,activation="softmax"))

In [ ]:
# 합성곱 신경망을 구성한 이후, 모델 학습과정을 설정하는 부분.
# 여기서는 학습 이전에 학습과 관련된 설정을 수행하며, 특히 손실 함수와 최적화 방법을 정의함.

# 문제의 유형, 출력층의 활성화 함수, 손실 함수는 서로 연계되어 있음.
# 본 데이터는 Cat과 Dog을 분류하는 Binary Classification Problem Type임.
# 그에 따라 출력층의 활성화 함수는 Sigmoid, 손실 함수는 binary_crossentropy를 사용함.

# 옵티마이저는 케라스 모델의 컴파일에 필요한 또다른 요소이며, 여러 종류가 있음.
# 대부분의 CNN 기반 이미지 분류 문제에서 rmsprop을 사용하는데 이유는 당장 알 수 없음.

# 메트릭은 평가 기준을 말하며, 일반적인 평가 기준으로 accuracy를 사용함.
# 손실 함수와 함꼐 훈련 과정 모니터링에 사용되는 두 지표임.
model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 126, 126, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 63, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 61, 61, 64)       2

In [ ]:
# 간혹 딥러닝 도중 과적합이 의심되는 경우, 콜백 함수(Callback Function)를 활용해 이에 대처함.
# 콜백 함수 : 함수 내에 또다른 함수가 들어 있는 경우, 어떤 함수를 사용할지를 인자로 지정하는 형태의 함수.

# EarlyStopping은 학습 진행 도중 손실 함수가 더 이상 줄어들지 않는 조건에서 실행되며 학습을 자동으로 중단함.
# ReduceLROnPlateau는 학습 진행 도중 평가 지표가 더 이상 향상되지 않는 조건에서 실행되며 학습률을 자동으로 조정함.
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# EarlyStopping의 patience = 10은 10개의 에포크 동안 손실 함수가 감소하지 않으면 학습을 종료하는 파라미터.
earlystop = EarlyStopping(patience=10)

learning_rate_reduction=ReduceLROnPlateau(
    # 현재 학습률을 조정하기 위해 모니터링하는 대상은 성과 지표임. val_loss로 손실 함수를 볼 수 있음.
                        monitor= "val_accuracy", 
                        # 지정한 수만큼의 기간에서 성능 향상이 일어나지 않으면 학습률을 조정함.
                        patience = 2, 
                        # 학습률 조정에 사용되는 새로운 값으로, factor * 기존 학습률 = 새 학습률이 됨.
                        factor = 0.5, 
                        # 학습률의 하한선으로, 이보다 낮은 학습률로 조정되지 않음.
                        min_lr=0.0001,
                        # verbose = 1이면 에포크마다 학습 진행사항을 출력하며, 0이면 과정을 출력하지 않음.
                        verbose=1)
                        
callbacks = [earlystop, learning_rate_reduction]

In [ ]:
# 이미지 제네레이터에서 class_mode = "categorical"로 지정하기 위해 컬럼 카테고리를 스트링으로 변경함. 
df['category']=df['category'].replace({0:'cat',1:"dog"})

In [ ]:
# 본격적인 훈련을 위해 데이터를 구분하는 부분. 여기서는 평가용 데이터와 별개로 훈련용과 검증용 데이터를 구분함.
# Sklearn에서 제공하는 train_test_split()은 비율로 데이터를 자를 수 있으며, test_size = 0.2는 8:2 분할을 의미함.
# random_state = 42는 분할 시 시드값.
train_df, validate_df = train_test_split(df , test_size=0.2, random_state= 42)

# Pandas에서 제공하는 reset_index() 메서드는 인덱스를 새롭게 지정하며, 동시에 drop = True로 기존의 구조를 그대로 유지함.
# drop = False면 Index로 사용된 기존의 무언가가 열 안에 여전히 남아 있게 됨.
train_df=train_df.reset_index(drop=True)
validate_df=validate_df.reset_index(drop=True)


train_df['category'].value_counts()


cat    365
Name: category, dtype: int64

In [ ]:

validate_df['category'].value_counts()


cat    92
Name: category, dtype: int64

In [ ]:
# 합성곱 신경망의 성능을 높이기 위한 방법으로 자주 사용되는 Data Augmentation를 다루는 부분.
# 훈련용 데이터의 수가 모자라거나 그 특성을 충분히 반영하지 못하고 있다고 생각되는 경우 성능을 크게 높일 수 있음.

# shape() 메서드는 행과 열을 리스트 형태로 출력하며, 이 중 행의 갯수를 total_ 변수로 함. 즉 총 데이터 수를 취함.
total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=15

# ImageDataGenerator() 함수가 Augmentation의 대부분의 과정을 제공하므로 파라미터를 입력하는 것으로 데이터를 부풀릴 수 있음.
# 아래의 부분은 생성 규칙이므로, 그에 맞게 제너레이터를 새로 만들어줘야 함.
train_datagen=ImageDataGenerator(
        # rotation_range는 지정된 각도 범위 내에서 임의로 원본 이미지를 회전시킨 데이터를 생성함.
        rotation_range=15,
        # 원본 이미지는 0 ~ 255의 RGB 계수로 구성되어 있으므로, 통상적인 Learning Rate에 비해 과하게 높음.
        # 따라서 이를 1/255로 Scaling하여 0 ~ 1 사이의 값을 갖게 함. 다른 전처리보다 우선적으로 적용됨.
        rescale=1./255,
        # shear_range는 시계 반대방향으로 지정된 라디안만큼 원본 데이터를 변형시킨 데이터를 생성함.
        shear_range=0.1,
        # zoom_range는 지정된 확대/축소 범위만큼 원본 이미지를 1+a배 확대 / 1-a배 축소한 데이터를 생성함,
        zoom_range=0.2,
        # horizontal_flip은 수평 방향으로 원본 이미지를 반전시킨 데이터를 생성함.
        horizontal_flip=True,
        # width_shift_range는 지정된 범위 내에서 수평 방향으로 원본 이미지를 이동시킨 데이터를 생성함.
        width_shift_range=0.1, 
        # height_shift_range는 지정된 범위 내에서 수직 방향으로 원본 이미지를 이동시킨 데이터를 생성함.
        height_shift_range=0.1)

# 생성 규칙에 집어넣을 원본 데이터에 기반한 제네레이터를 생성함. flow_from_dataframe은 데이터 프레임에서 파일을 불러올 때 사용함.
# (아직 데이터가 올라와 있지 않은 경우 flow_from_directory를 사용할 수 있음.)
train_generator=train_datagen.flow_from_dataframe(
    train_df, # 데이터 프레임 이름
    "/content/gdrive/My Drive/Colab Notebooks/catdog/train/train/", # 데이터 위치
    x_col = "filename", # 파일 열 이름
    y_col = "category", # 클래스 열 이름
    target_size = IMAGE_SIZE, # 이미지 사이즈
    class_mode = "categorical", # Y값 변환
    # Batch Size는 몇 개의 예측값마다 실제값과 비교하여 가중치를 수정할지 정함.
    batch_size = batch_size ) # 배치 사이즈
    
# 검증용 데이터는 실제 세상의 이미지를 그대로 반영해야 하므로, RGB 정규화 이외에 다른 조치는 취하지 않음.
validate_datagen=ImageDataGenerator(rescale=1./255)


validation_generator=validate_datagen.flow_from_dataframe(
    validate_df,
  "/content/gdrive/My Drive/Colab Notebooks/catdog/train/train/",
    x_col= "filename",
    y_col= "category",
    target_size = IMAGE_SIZE,
    class_mode = "categorical",
    batch_size = batch_size )

Found 365 validated image filenames belonging to 1 classes.
Found 92 validated image filenames belonging to 1 classes.


In [ ]:
# 데이터 확인인가? 왜 여기 있지?
example_df=train_df.sample(n=1).reset_index(drop=True)
example_df

,filename,category
0,cat.10018.jpg,cat


In [ ]:
# 데이터 확인인가? 왜 여기 있지?
example_generator = train_datagen.flow_from_dataframe(
                    example_df,
                    "/content/gdrive/My Drive/Colab Notebooks/catdog/train/train/",
                    x_col = "filename",
                    y_col = "category",
                    target_size = IMAGE_SIZE,
                    class_mode = "categorical")

Found 1 validated image filenames belonging to 1 classes.


In [ ]:
# 데이터 확인인가? 왜 여기 있지?
plt.figure(figsize=(10,10))
for i in range(0,15):
    plt.subplot(5,3,i+1)
    for xBatch, yBatch in example_generator:
        image = xBatch[0]
        plt.imshow(image)
        break
plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

In [ ]:
# 모델과 데이터 설정이 끝났으므로 본격적으로 모델을 학습시키는 부분.

# 앞서 다룬 Batch Size는 n개의 예측값마다 실제값과 비교하여 가중치를 수정함.
# 그러므로, Batch Size가 클수록 파일 용량이 크고 학습이 천천히 일어남, 작을수록 분석 시간이 길어지고 학습이 빠르게 일어남.
# epoch는 하나의 데이터셋을 몇 회 반복 학습할지 결정하는 부분임.
# 같은 데이터셋이라도 가중치가 계속 달라지고 있으므로 추가적인 학습이 가능함.
# 너무 많은 Epoch는 과적합(Overfitting)을 야기할 수 있어 EarlyStopping과 같은 Callback Function이 필요함.
epochs = 3 

# 앞서 합성곱 신경망 모델을 model에 할당했음.
# 일반적인 모델 적합 시에는 fit() 메서드를 사용하지만, 여기서는 ImageDataGenerator라는 제네레이터를 이용함.
# 따라서 제네레이터로 이미지를 담은 배치로 학습시키므로, fit_generator() 메서드를 사용함.
history = model.fit_generator(
    # 훈련 데이터셋을 제공할 제네레이터를 지정함.
    train_generator, 
    # 전체 데이터셋에 대한 학습 반복 횟수를 지정함.
    epochs = epochs, 
    # 한 Epoch마다 학습 반복 횟수를 지정함. 주로 전체 훈련 데이터 수 / 배치 사이즈로 지정됨.
    steps_per_epoch = total_train//batch_size ,
    # 검증용 데이터셋을 제공할 제네레이터를 지정함.
    validation_data=  validation_generator,
    # 한 epoch 종료마다 검증할 때 사용되는 검증 반복 횟수를 지정함. 주로 전체 검증 데이터 수 / 배치 사이즈로 지정됨.
    validation_steps = total_validate//batch_size,
    # 손실 함수가 감소하지 않거나, 학습률이 더 증가하지 않을 경우를 대비한 Callback Function을 선언함.
    callbacks = callbacks,
)

# 케라스는 save() 함수로 모델 아키텍쳐와 가중치를 h5 확장자 형식으로 저장할 수 있음.
model.save_weights("model.h5")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if __name__ == '__main__':


Epoch 1/3
24/24 [==============================] - 71s 3s/step - loss: 2.8809 - accuracy: 0.4943 - val_loss: 1.7612 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 2/3
24/24 [==============================] - 21s 873ms/step - loss: 2.3841 - accuracy: 0.4971 - val_loss: 4.9100 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/3
24/24 [==============================] - 21s 856ms/step - loss: 2.2433 - accuracy: 0.4886 - val_loss: 2.0783 - val_accuracy: 1.0000 - lr: 0.0010


In [ ]:
# 히스토리 객체는 model.fit()의 리턴값으로 Loss와 Accuracy에 관한 정보를 저장하고 있음.
historyDict=history.history

# acc : 매 Epoch마다의 훈련 정확도
acc=history.history['accuracy']
# val_acc : 매 Epoch마다의 검증 정확도
val_acc=history.history['val_accuracy']
# loss : 매 Epoch마다의 훈련 손실값
loss=history.history['loss']
# val_loss : 매 Epoch마다의 검증 손실값
val_loss=history.history['val_loss']

%matplotlib notebook
epo = range(1, len(acc)+1)
plt.plot(epo, loss, 'bo', label="Training loss")
plt.plot(epo, val_loss, 'b', label="Val loss")
plt.xlabel("epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

In [ ]:
%matplotlib notebook

plt.plot(epo, acc, 'ro', label="Training accuracy")
plt.plot(epo, val_acc, 'r', label="Val accuracy")
plt.xlabel("epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

In [ ]:
# 평가용 데이터 역시 훈련용 데이터와 같은 방식으로 불러옴. 단 여기에는 1과 0을 찾는 조치를 취하지 않음.

filenames = os.listdir("/content/gdrive/My Drive/Colab Notebooks/catdog/test1/test1/")

test_df=pd.DataFrame( {"filename":filenames} )      
test_df

nbsamples=test_df.shape[0]

In [ ]:
# 평가용 데이터를 위한 제네레이터를 만드는 부분, 검증용 데이터와 마찬가지로 RGB값 조정 외의 조치를 취하지 않음.

test_datagen=ImageDataGenerator(rescale=1./255)
test_generator=test_datagen.flow_from_dataframe(
    test_df,
    "/content/gdrive/My Drive/Colab Notebooks/catdog/test1/test1/",
    x_col= "filename",
    y_col= None,
    target_size = IMAGE_SIZE,
    class_mode = None,
    batch_size = batch_size,
    shuffle = False)

Found 6597 validated image filenames.


In [ ]:
# 평가용 데이터와 모델에 기반한 예측을 수행함.
# predict_generator() 메서드는 fit_generator() 메서드처럼 제네레이터를 통해 생성될 데이터를 기준으로 함.

predict=model.predict_generator(test_generator, 
                                steps=nbsamples/batch_size, 
                                callbacks=callbacks)

# 앞서 이미지 제네레이터에 categorical을 적용하기 위해 0과 1을 cat, dog으로 변환했음에 유의함.
# 앞선 분류에서 출력층의 예측값은 0과 1 사이의 값으로 제시되었으며, Softmax 활성화 함수를 거쳐 0 또는 1로 나타남.
test_df['category']=np.argmax(predict, axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  after removing the cwd from sys.path.


In [ ]:
# 예측값과 실제값의 적절한 비교를 위해 0을 cat으로, 1을 dog으로 변환하는 과정을 거침.

test_df['category']=test_df['category'].replace({0:'cat',1:"dog"})
ex_df=test_df.sample(n=1).reset_index(drop=True)
ex_df

,filename,category
0,1894.jpg,cat


In [ ]:
ex_generator = test_datagen.flow_from_dataframe(
                    ex_df,
                    "/content/gdrive/My Drive/Colab Notebooks/catdog/test1/test1/",
                    x_col = "filename",
                    y_col = None,
                    target_size = IMAGE_SIZE,
                    class_mode = None)

Found 1 validated image filenames.


In [ ]:
test_sample=list(ex_df.filename)

sample = ""
for test in test_sample:
    sample += test

In [ ]:
image = load_img("/content/gdrive/My Drive/Colab Notebooks/catdog/test1/test1/"+sample)
plt.figure(figsize=(8,8))
plt.imshow(image)

plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>